In [38]:
import pickle 
import sys
import pandas as pd 
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import torch 
import torch.nn as nn

# loading file
with open('../components/review_embedding.pkl', 'rb') as handle:
    x = pickle.load(handle)

In [14]:
df = pd.DataFrame(x)

In [33]:
class ReviewDataSet(Dataset):
    def __init__(self, reviews, ratings):
        self.reviews = reviews 
        self.ratings = ratings

    def __len__(self):
        return len(self.ratings)
    
    def __getitem__(self, index):
        reviews = self.reviews[index]
        ratings = self.ratings[index]
        
        return reviews, int(ratings)

batch_size = 2
train_set = ReviewDataSet(reviews=df.embedding, ratings=df.overall)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=0)

In [34]:
for ratings, labels in train_loader:
    print('Size of image:', ratings.size())  # batch_size * 3 * 224 * 224
    print('Type of image:', ratings.dtype)   # float32
    print('Size of label:', ratings.size())  # batch_size
    print('Type of label:', ratings.dtype)   # int64(long)
    break

Size of image: torch.Size([2, 768])
Type of image: torch.float32
Size of label: torch.Size([2, 768])
Type of label: torch.float32


In [39]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        self.fc_layers = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(256, 1)
        )
    
    def forward(self, x):
        pred_ratings = self.fc_layers(x)
        return pred_ratings

n_epochs = 30
lr = 0.0005
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss() 

device = "cuda" 
model = Classifier().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


for epoch in range(n_epochs): 

    # training mode
    model.train()
    train_loss = []
    train_accs = []


    for batch in tqdm(train_loader):
        reviews, rating = batch 

        logits = model(reviews.to(device))
        loss = criterion(logits, rating.float().to(device))

        optimizer.zero_grad()
        loss.backward()
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        optimizer.step()

        # acc = (logits.argmax(dim=-1) == rating.float().to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)
    
    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)
    
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")


  0%|          | 0/114484 [00:00<?, ?it/s]C:\Users\Stevie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|█████████▉| 114442/114484 [04:26<00:00, 422.88it/s]C:\Users\Stevie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████|

[ Train | 001/030 ] loss = 1.51623, acc = 0.00000


  9%|▊         | 9789/114484 [00:21<03:45, 463.89it/s]


KeyboardInterrupt: 

In [40]:
rating

tensor([4, 3])

In [41]:
logits

tensor([[4.0907],
        [3.7234]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [45]:
reviews

tensor([[ 0.3894, -0.0420,  0.4633,  ..., -0.1228,  0.1068,  0.0121],
        [-0.1157,  0.0127,  0.5306,  ..., -0.1855, -0.1346,  0.0293]])

In [46]:
batch

[tensor([[ 0.3894, -0.0420,  0.4633,  ..., -0.1228,  0.1068,  0.0121],
         [-0.1157,  0.0127,  0.5306,  ..., -0.1855, -0.1346,  0.0293]]),
 tensor([4, 3])]

In [47]:
logits.argmax(dim=-1)

tensor([0, 0], device='cuda:0')

In [ ]:
acc = (logits.argmax(dim=-1) == rating.float().to(device)).float().mean()